<a href="https://colab.research.google.com/github/venkataravuri/ai-ml-models/blob/main/rag_add_citations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Source

https://python.langchain.com/docs/how_to/qa_citations/

In [ ]:
%pip install -qU langchain langchain-openai langchain-anthropic langchain-community wikipedia

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You're a helpful AI assistant. Given a user question "
    + "and some Wikipedia article snippet, answer the user "
    + "question. If none of the articles answer the question, "
    + "just say you don't know."
    + "\n\nHere are the Wikipedia articles: "
    + "{context}"
)

retriever = WikipediaRetriever(top_k_results=6, doc_content_chars_max=2000)
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])
prompt.pretty_print()

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(
        context=retriever.with_fallbacks([format_docs]).stream
    )
    | prompt
    | llm
    | StrOutputParser()
)

retrieve_docs = (lambda x: x["input"]) | retriever

chain = RunnablePassthrough.assign(
    input=retrieve_docs
) | rag_chain_from_docs

In [ ]:
result = chain.invoke("input": "What is the capital of France?")
print(result.keys())
print(result["context"][0])
print(result["answer"])

In [ ]:
from pydantic import BaseModel, Field

claass CitedAnswer(BaseMode):
  """Answer the user question based only on the given sources, and cite the sources used."""

  answer: str = Field(
      ...,
      description="The answer to the user question, which is based only on the given sources."
  )
  citations: List[str] = Field(
      ...,
      description="The integer IDs of the SPECIFIC sources which justify the answer."
  )

In [ ]:
structured_llm = llm.with_structured_output(CitedAnswer)

example_q = """What Brain's height?

Source: 1
Information: Suzy is 6'2"

Souce: 2
Information: Jeremiah is blonde

Source: 3
Information: Brian is 3 inchers shorter than Suzy"""

result = structured_llm.invoke(example_q)
print(result.answer)
print(result.citations)

In [ ]:
def format_docs_with_id(docs: List[Document]) -> str:
  formatted = [
      f"Source ID: {i} \nArticle Title: {doc.metadata['title']}\nArticle Snippet: {doc.page_content}"
      for i, doc in enumerate(docs)
  ]

rag_chain_from_docs = (
    RunnablePassthrough.assign(
        context=(lamda x: format_docs_with_id(x["context"]))
    )
    | prompt
    | structured_llm
 )

retreive_docs = (lambda x: x["input"]) | retriever

chain = RunnablePassthrough.assign(
    context=retrieve_docs
) | rag_chain_from_docs

In [ ]:
result = chain.invoke("input": "How fast are cheethas?")
print(result.answer)
print(result.citations)